### 🧠 ¿Qué es la Descomposición de Consultas?

La **descomposición de consultas** es el proceso de tomar una pregunta compleja de múltiples partes y dividirla en sub-preguntas más simples y atómicas que pueden ser recuperadas y respondidas individualmente.

#### ✅ ¿Por qué usar Descomposición de Consultas?

**Problemas que resuelve:**
- Las consultas complejas a menudo involucran múltiples conceptos
- Los LLMs o recuperadores pueden perder partes de la pregunta original
- Permite razonamiento multi-hop (responder en pasos)
- Posibilita paralelismo (especialmente en frameworks multi-agente)

**Ejemplo:**
- **Consulta compleja:** "¿Cómo usa LangChain la memoria y los agentes comparado con CrewAI?"
- **Descomposición:**
  1. ¿Qué mecanismos de memoria ofrece LangChain?
  2. ¿Cómo funcionan los agentes en LangChain?
  3. ¿Qué mecanismos de memoria ofrece CrewAI?
  4. ¿Cómo funcionan los agentes en CrewAI?
  5. ¿Cuáles son las diferencias clave entre ambos?

**Ventajas:**
- ✅ Mejora la precisión en respuestas complejas
- ✅ Permite recuperación más enfocada para cada aspecto
- ✅ Facilita el razonamiento paso a paso
- ✅ Reduce sobrecarga de contexto en cada consulta
- ✅ Permite procesamiento paralelo de sub-preguntas

In [1]:
# Importación de librerías necesarias para el pipeline RAG con descomposición de consultas

# init_chat_model: Para inicializar modelos de chat de diferentes proveedores (OpenAI, Groq, etc.)
from langchain.chat_models import init_chat_model

# PromptTemplate: Para crear plantillas de prompts con variables dinámicas
from langchain.prompts import PromptTemplate

# TextLoader: Para cargar archivos de texto plano
from langchain.document_loaders import TextLoader

# RecursiveCharacterTextSplitter: Para dividir texto en fragmentos (chunks) manejables
from langchain.text_splitter import RecursiveCharacterTextSplitter

# HuggingFaceEmbeddings: Para generar embeddings vectoriales usando modelos de HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings

# FAISS: Biblioteca de Facebook para búsqueda de similitud vectorial eficiente
from langchain_community.vectorstores import FAISS

# StrOutputParser: Para convertir la salida del LLM a string
from langchain_core.output_parsers import StrOutputParser

# create_stuff_documents_chain: Crea una cadena que inserta documentos en un prompt
from langchain.chains.combine_documents import create_stuff_documents_chain

# RunnableSequence: Permite encadenar operaciones secuencialmente
from langchain_core.runnables import RunnableSequence

c:\Udemy\RAGBootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paso 1: Cargar y crear el almacén vectorial del documento

# Cargar el archivo de texto que contiene información sobre LangChain y CrewAI
loader = TextLoader("langchain_crewai_dataset.txt")

# load() devuelve una lista de documentos con todo el contenido del archivo
docs = loader.load()

# Crear un splitter para dividir el texto en fragmentos más pequeños
# chunk_size=300: Cada fragmento tendrá máximo 300 caracteres
# chunk_overlap=50: Habrá una superposición de 50 caracteres entre fragmentos consecutivos
# La superposición ayuda a mantener el contexto entre fragmentos adyacentes
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# Dividir los documentos en fragmentos (chunks) manejables
chunks = splitter.split_documents(docs)

# Inicializar el modelo de embeddings de HuggingFace
# all-MiniLM-L6-v2 es un modelo compacto y eficiente que genera vectores de 384 dimensiones
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Crear el almacén vectorial FAISS a partir de los fragmentos de documentos
# FAISS convierte cada fragmento en un vector numérico para búsqueda de similitud eficiente
vectorstore = FAISS.from_documents(chunks, embedding)

# Crear el recuperador con MMR (Maximal Marginal Relevance)
# search_type="mmr": Usa MMR para balancear relevancia y diversidad
# k=4: Recuperar los 4 documentos más relevantes
# lambda_mult=0.7: Factor que controla el balance entre relevancia (1.0) y diversidad (0.0)
#                  0.7 significa 70% relevancia, 30% diversidad
retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 4, "lambda_mult": 0.7}
)

In [3]:
# Paso 2: Configurar el LLM (Large Language Model) y variables de entorno

# Importar módulos para manejo de variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
# Este archivo contiene las API keys de forma segura
load_dotenv()

# Establecer la API key de Groq desde las variables de entorno
# Groq ofrece inferencia de LLM ultra-rápida
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Inicializar el LLM usando el modelo Gemma2-9B-IT de Groq
# Gemma2 es un modelo eficiente de Google, optimizado para tareas de razonamiento
# "it" significa "instruction-tuned" (ajustado para seguir instrucciones)
llm = init_chat_model(model="groq:llama-3.1-8b-instant")

# Mostrar la configuración del LLM
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002A7A20EB0B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002A7A3978800>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
# Paso 3: Crear la cadena de descomposición de consultas

# Esta plantilla de prompt instruye al LLM para descomponer consultas complejas
# El objetivo es dividir una pregunta compleja en 2-4 sub-preguntas más simples
# Cada sub-pregunta debe ser atómica (enfocada en un solo aspecto)
decomposition_prompt = PromptTemplate.from_template("""
Eres un asistente de IA. Descompón la siguiente pregunta compleja en 2 a 4 sub-preguntas más simples para mejorar la recuperación de documentos.

Pregunta: "{question}"

Sub-preguntas:
""")

# Crear la cadena de descomposición de consultas
# Esta cadena conecta: prompt → LLM → parser de salida
# El operador | (pipe) encadena los componentes secuencialmente
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [6]:
# Paso 4: Probar la descomposición de una consulta compleja

# Definir una consulta compleja que involucra múltiples conceptos:
# 1. Memoria en LangChain
# 2. Agentes en LangChain
# 3. Memoria en CrewAI
# 4. Agentes en CrewAI
# 5. Comparación entre ambos
query = "¿Cómo usa LangChain la memoria y los agentes comparado con CrewAI?"

# Invocar la cadena de descomposición para dividir la consulta compleja
# El LLM generará 2-4 sub-preguntas más específicas y enfocadas
decomposition_question = decomposition_chain.invoke({"question": query})

In [7]:
# Mostrar las sub-preguntas generadas por el LLM
# Observa cómo el LLM divide la consulta compleja en preguntas atómicas
# Cada sub-pregunta se enfoca en un aspecto específico
print("🔍 Sub-preguntas Generadas:")
print(decomposition_question)

🔍 Sub-preguntas Generadas:
Para mejorar la recuperación de documentos, puedo descomponer la pregunta original en 4 sub-preguntas más simples. Aquí te presento la descomposición:

1. **¿Qué es LangChain y qué tipo de memoria utiliza?**
 - Esta sub-pregunta busca entender la arquitectura fundamental de LangChain y cómo se relaciona con la memoria.

2. **¿Cómo funcionan los agentes en LangChain?**
 - Esta sub-pregunta se enfoca en la funcionalidad de los agentes en LangChain y cómo interactúan con la memoria.

3. **¿Qué es CrewAI y qué tipo de memoria utiliza?**
 - Esta sub-pregunta busca entender la arquitectura de CrewAI y cómo se relaciona con la memoria, para poder compararla con LangChain.

4. **¿Cómo comparar la implementación de memoria y agentes en LangChain y CrewAI?**
 - Esta sub-pregunta busca identificar los aspectos clave para comparar la implementación de memoria y agentes en ambos frameworks.

Al responder estas sub-preguntas, podríamos obtener una comprensión más detallada

In [8]:
# Paso 5: Crear la cadena de Q&A para cada sub-pregunta

# Esta plantilla define cómo el LLM usará el contexto recuperado para responder
# Se aplicará individualmente a cada sub-pregunta
qa_prompt = PromptTemplate.from_template("""
Usa el contexto a continuación para responder la pregunta.

Contexto:
{context}

Pregunta: {input}
""")

# Crear la cadena de documentos que combina el LLM con el prompt de Q&A
# Esta cadena toma documentos recuperados y los inserta en el prompt
# "stuff" significa que todos los documentos se insertan directamente en el prompt
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [9]:
# Paso 6: Construir el pipeline RAG completo con descomposición de consultas

def full_query_decomposition_rag_pipeline(user_query):
    """
    Pipeline RAG completo que implementa descomposición de consultas.
    
    Flujo del pipeline:
    1. Descompone la consulta compleja en sub-preguntas
    2. Para cada sub-pregunta:
       a. Recupera documentos relevantes del vector store
       b. Genera una respuesta usando los documentos como contexto
    3. Combina todas las respuestas en un resultado final
    
    Args:
        user_query (str): La consulta compleja del usuario
    
    Returns:
        str: Respuestas combinadas para todas las sub-preguntas
    """
    
    # 1. Descomponer la consulta compleja en sub-preguntas
    # El LLM genera un texto con múltiples sub-preguntas separadas por saltos de línea
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    
    # 2. Parsear el texto de sub-preguntas en una lista limpia
    # - split("\n"): Divide el texto por saltos de línea
    # - strip("-•1234567890. "): Elimina caracteres de numeración y viñetas
    # - if q.strip(): Filtra líneas vacías
    sub_questions = [
        q.strip("-•1234567890. ").strip() 
        for q in sub_qs_text.split("\n") 
        if q.strip()
    ]
    
    # 3. Lista para almacenar los resultados de cada sub-pregunta
    results = []
    
    # 4. Procesar cada sub-pregunta individualmente
    for subq in sub_questions:
        # a. Recuperar documentos relevantes para esta sub-pregunta específica
        #    El recuperador usa MMR para obtener documentos relevantes y diversos
        docs = retriever.invoke(subq)
        
        # b. Generar una respuesta usando los documentos recuperados como contexto
        #    La cadena qa_chain inserta los documentos en el prompt y llama al LLM
        result = qa_chain.invoke({"input": subq, "context": docs})
        
        # c. Formatear y almacenar el resultado con la pregunta y respuesta
        results.append(f"Q: {subq}\nA: {result}")
    
    # 5. Combinar todas las respuestas en un solo texto
    # Separar cada par Q&A con dos saltos de línea para mejor legibilidad
    return "\n\n".join(results)

In [10]:
# Paso 7: Ejecutar el pipeline completo con una consulta compleja

# Definir la consulta compleja que involucra comparación entre dos frameworks
query = "¿Cómo usa LangChain la memoria y los agentes comparado con CrewAI?"

# Invocar el pipeline RAG completo con descomposición de consultas
# El pipeline automáticamente:
# 1. Descompondrá la consulta en sub-preguntas
# 2. Recuperará documentos para cada sub-pregunta
# 3. Generará respuestas individuales
# 4. Combinará todas las respuestas
final_answer = full_query_decomposition_rag_pipeline(query)

# Mostrar el resultado final con todas las sub-preguntas y sus respuestas
print("✅ Respuesta Final:")
print("="*80)
print(final_answer)

✅ Respuesta Final:
Q: Excelente pregunta. Para descomponerla en sub-preguntas más simples, te propongo las siguientes:
A: No hay una pregunta específica en el contexto proporcionado. Parece que estás presentando información sobre LangChain y CrewAI, que son herramientas relacionadas con la inteligencia artificial y los lenguajes de marcado natural (LLM). Si te gustaría formular una pregunta sobre este tema, estaré encantado de ayudarte a descomponerla en sub-preguntas más simples. ¿Cuál es tu pregunta?

Q: **¿Qué es LangChain y qué tipo de memoria utiliza?**
A: LangChain es una plataforma para desarrollar aplicaciones con Inteligencia Artificial de Lenguaje (LLM) escalables y fáciles de mantener. 

LangChain utiliza dos tipos de memoria: 

- **ConversationBufferMemory**: que permite que el LLM mantenga la información de los turnos de conversación anteriores.
- **ConversationSummaryMemory**: que resume interacciones largas para ajustarse a los límites de tokens.

Q: * Esta sub-pregunta 

### 📊 Resumen: Beneficios de la Descomposición de Consultas

**Flujo de Descomposición:**
```
Consulta Compleja
       ↓
Descomposición (LLM)
       ↓
Sub-pregunta 1 → Recuperación → Respuesta 1
Sub-pregunta 2 → Recuperación → Respuesta 2
Sub-pregunta 3 → Recuperación → Respuesta 3
Sub-pregunta 4 → Recuperación → Respuesta 4
       ↓
Respuesta Final Combinada
```

**Ventajas clave:**
- ✅ **Precisión mejorada**: Cada sub-pregunta obtiene contexto específico
- ✅ **Razonamiento multi-hop**: Permite responder preguntas que requieren múltiples pasos
- ✅ **Mejor cobertura**: No se pierden aspectos de la pregunta original
- ✅ **Recuperación enfocada**: Cada búsqueda es más específica y relevante
- ✅ **Respuestas estructuradas**: El resultado final está organizado por aspectos

**Casos de uso ideales:**
- ✅ Preguntas comparativas ("A vs B")
- ✅ Consultas multi-aspecto (memoria + agentes + herramientas)
- ✅ Preguntas que requieren razonamiento en pasos
- ✅ Análisis complejos que involucran múltiples entidades
- ✅ Investigación exploratoria de temas amplios

**Consideraciones:**
- ⚠️ **Latencia aumentada**: Múltiples llamadas al LLM (1 descomposición + N respuestas)
- ⚠️ **Costo mayor**: Más tokens consumidos por las llamadas adicionales
- ⚠️ **Complejidad**: Requiere parseo y manejo de múltiples sub-preguntas
- ⚠️ **No siempre necesario**: Para consultas simples, puede ser overkill

**Comparación con Query Expansion:**

| Aspecto | Query Expansion | Query Decomposition |
|---------|----------------|---------------------|
| **Propósito** | Enriquecer consulta con sinónimos | Dividir consulta en partes |
| **Llamadas LLM** | 1 expansión + 1 respuesta | 1 descomposición + N respuestas |
| **Mejor para** | Consultas vagas o técnicas | Consultas complejas multi-aspecto |
| **Latencia** | Baja-Media | Media-Alta |
| **Precisión** | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ |

**¿Cuándo usar cada técnica?**
- **Query Expansion**: "memoria en LangChain" → agregar sinónimos técnicos
- **Query Decomposition**: "¿Cómo se compara LangChain con CrewAI en memoria y agentes?" → dividir en sub-preguntas específicas
- **Ambas**: Para máxima precisión, primero descomponer y luego expandir cada sub-pregunta